In [1]:
from process import ProcessADS, ProcessGraph

DEV_KEY = "kNUoTurJ5TXV9hsw9KQN1k8wH4U0D7Oy0CJoOvyw"


### Data Warehousing API for NASA ADS

These two classes from the process module get our data out of NASA ADS and into a table format suitable for warehousing on our server (SQLite database).

The ProcessADS class takes a string query q, a dev key for the ADS api, and an integer value for max_pages (1 to 5 seem to work the best). In this way we can create multiple instances for each query in the same python script/session.


In [2]:
p = ProcessADS(q='stars',key=DEV_KEY,max_pages=2)


/home/bsnacks/Documents/Flask_projects/venv_nasaflask/lib/python2.7/site-packages/ads/utils.py:23: UserWarning: You are lazy loading attributes via 'author', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,
/home/bsnacks/Documents/Flask_projects/venv_nasaflask/lib/python2.7/site-packages/ads/utils.py:23: UserWarning: You are lazy loading attributes via 'pub', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,


We can investigate the results of the query by checking out the nodes and edges. These are just pandas dataframes and respond to all the df methods. 



In [3]:
p.nodes.head()

,label,node_type,id
0,"Monkhorst, Hendrik J.",Author,0
1,"Pack, James D.",Author,1
2,"Schlegel, David J.",Author,2
3,"Finkbeiner, Douglas P.",Author,3
4,"Davis, Marc",Author,4


The fields that store the tables in both Process classes are private, so they cannot be set directly (and accidentally overwritten). If you want to manipulate the dataframes for whatever reason it needs to be stored to a new variable. Don't do anything that changes the dfs in place inside the objects!!


In [4]:
#p.edges = None  # direct setting throws an error

ProcessGraph processes the query object with networkx and delivers 3 sets of graphs in node and edgelist dataframes. Main is the large main query graph, lg_cc_subgraph are the largest bipartite connected components and islands are bipartite subgraphs with the highest edgeweight for each node type. Betweenness centrality, degree centrality, and pagerank for all nodes in each graph are also added. These will play a factor in the visualization's node size in sigmajs.

The dataframes you can look at (assuming graphs = ProcessGraph(p):
* graphs.main_nodes
* graphs.main_edges
* graphs.lg_cc_nodes
* graphs.lg_cc_edges
* graphs.islands_nodes
* graphs.islands_edges

You can also check out the 3 graph networkx graph objects:
* graphs.g   (main graph)
* graphs.lg_cc_subgraph
* graphs.islands_subgraph



In [5]:
graphs = ProcessGraph(p)

graphs.main_nodes.head()

,id,label,node_type,zbetween_central,zdeg_central,zpagerank
0,0,"Monkhorst, Hendrik J.",Author,0.0,0.002564,0.001579
1,1,"Pack, James D.",Author,0.0,0.002564,0.001579
2,2,"Schlegel, David J.",Author,0.0,0.002564,0.001301
3,3,"Finkbeiner, Douglas P.",Author,0.0,0.002564,0.001301
4,4,"Davis, Marc",Author,0.0,0.002564,0.001301


In [7]:
graphs.a_lg_cc_subgraph.nodes(data=True)[0:5]

[(2,
  {'id': 2,
   'label': u'Schlegel, David J.',
   'node_type': 'Author',
   'zbetween_central': 0.0,
   'zdeg_central': 0.5358851674641149,
   'zpagerank': 0.0051597816207715405}),
 (3,
  {'id': 3,
   'label': u'Finkbeiner, Douglas P.',
   'node_type': 'Author',
   'zbetween_central': 0.0,
   'zdeg_central': 0.5358851674641149,
   'zpagerank': 0.0051597816207715405}),
 (4,
  {'id': 4,
   'label': u'Davis, Marc',
   'node_type': 'Author',
   'zbetween_central': 0.0,
   'zdeg_central': 0.5358851674641149,
   'zpagerank': 0.0051597816207715405}),
 (5,
  {'id': 5,
   'label': u'Perlmutter, S.',
   'node_type': 'Author',
   'zbetween_central': 0.0,
   'zdeg_central': 0.5358851674641149,
   'zpagerank': 0.0051597816207715405}),
 (6,
  {'id': 6,
   'label': u'Aldering, G.',
   'node_type': 'Author',
   'zbetween_central': 0.0,
   'zdeg_central': 0.5358851674641149,
   'zpagerank': 0.0051597816207715405})]

In [8]:
graphs.islands_graph.nodes(data=True)[:5]

[(141,
  {'id': 141,
   'label': u'Tremaine, Scott',
   'node_type': 'Author',
   'zbetween_central': 0.0,
   'zdeg_central': 0.7619047619047619,
   'zpagerank': 0.045454545454545435}),
 (38,
  {'id': 38,
   'label': u'Riess, Adam G.',
   'node_type': 'Author',
   'zbetween_central': 0.0,
   'zdeg_central': 0.7619047619047619,
   'zpagerank': 0.045454545454545435}),
 (39,
  {'id': 39,
   'label': u'Filippenko, Alexei V.',
   'node_type': 'Author',
   'zbetween_central': 0.0,
   'zdeg_central': 0.7619047619047619,
   'zpagerank': 0.045454545454545435}),
 (40,
  {'id': 40,
   'label': u'Challis, Peter',
   'node_type': 'Author',
   'zbetween_central': 0.0,
   'zdeg_central': 0.7619047619047619,
   'zpagerank': 0.045454545454545435}),
 (46,
  {'id': 46,
   'label': u'Jha, Saurabh',
   'node_type': 'Author',
   'zbetween_central': 0.0,
   'zdeg_central': 0.7619047619047619,
   'zpagerank': 0.045454545454545435})]

If everything looks good, two methods are called to export the graphs to csv files. A subdirectory of the current working directory called csvs is created for the output. For our web app these csvs will get piped into a sqlite database that will sit on the server, but we can use these csvs for other graph APIs and databases if we wanted to...

In [9]:
graphs.export_main_to_csv()

In [10]:
graphs.export_subgraphs_to_csv()